In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window as W

# Initialize SparkSession
spark = SparkSession.builder.appName("leetcode_basic_joins").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 17:48:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
1. Customer Who Visited but Did Not Make Any Transactions
https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transactions/description/?envType=study-plan-v2&envId=top-sql-50 

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.
Input: 

Visits
+----------+-------------+
| visit_id | customer_id |
+----------+-------------+
| 1        | 23          |
| 2        | 9           |
| 4        | 30          |
| 5        | 54          |
| 6        | 96          |
| 7        | 54          |
| 8        | 54          |
+----------+-------------+
Transactions
+----------------+----------+--------+
| transaction_id | visit_id | amount |
+----------------+----------+--------+
| 2              | 5        | 310    |
| 3              | 5        | 300    |
| 9              | 5        | 200    |
| 12             | 1        | 910    |
| 13             | 2        | 970    |
+----------------+----------+--------+

Output: 
+-------------+----------------+
| customer_id | count_no_trans |
+-------------+----------------+
| 54          | 2              |
| 30          | 1              |
| 96          | 1              |
+-------------+----------------+

In [ ]:
visits_data = [(1, 23), (2, 9), (4, 30), (5, 54), (6, 96), (7, 54), (8, 54)]

transactions_data = [(2, 5, 310), (3, 5, 300), (9, 5, 200), (12, 1, 910), (13, 2, 970)]

# Create DataFrames
visits_df = spark.createDataFrame(visits_data, ["visit_id", "customer_id"])
transactions_df = spark.createDataFrame(
    transactions_data, ["transaction_id", "visit_id", "amount"]
)

In [13]:
(
    visits_df.join(
        transactions_df, on=visits_df.visit_id == transactions_df.visit_id, how="left"
    )
    .filter(F.col("transaction_id").isNull())
    .groupBy(F.col("customer_id"))
    .agg(F.count(visits_df.visit_id).alias("count_no_trans"))
    .show()
)

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         96|             1|
|         54|             2|
|         30|             1|
+-----------+--------------+



In [17]:
print(
    "Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday)."
)
data = [
    (1, "2015-01-01", 10),
    (2, "2015-01-02", 25),
    (3, "2015-01-03", 20),
    (4, "2015-01-04", 30),
]
weather_df1 = spark.createDataFrame(data, ["id", "recordDate", "temperature"])
weather_df2 = spark.createDataFrame(data, ["id", "recordDate", "temperature"])
print("Input:")
weather_df1.show()

print(
    """Output: 
+----+
| id |
+----+
| 2  |
| 4  |
+----+      
"""
)

Write a solution to find all dates' Id with higher temperatures compared to its previous dates (yesterday).
Input:


+---+----------+-----------+
| id|recordDate|temperature|
+---+----------+-----------+
|  1|2015-01-01|         10|
|  2|2015-01-02|         25|
|  3|2015-01-03|         20|
|  4|2015-01-04|         30|
+---+----------+-----------+

Output: 
+----+
| id |
+----+
| 2  |
| 4  |
+----+      



In [25]:
(
    weather_df1.join(
        weather_df2,
        how="inner",
        on=weather_df1.recordDate == F.date_add(weather_df2.recordDate, -1),
    )
    .filter(weather_df2.temperature > weather_df1.temperature)
    .select(weather_df2.id)
    .show()
)

+---+
| id|
+---+
|  4|
|  2|
+---+



In [30]:
print("""Average Time of Process per Machine
There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.
The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.
The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.
Input: """)
# Sample data
activity_data = [
    (0, 0, 'start', 0.712),
    (0, 0, 'end', 1.520),
    (0, 1, 'start', 3.140),
    (0, 1, 'end', 4.120),
    (1, 0, 'start', 0.550),
    (1, 0, 'end', 1.550),
    (1, 1, 'start', 0.430),
    (1, 1, 'end', 1.420),
    (2, 0, 'start', 4.100),
    (2, 0, 'end', 4.512),
    (2, 1, 'start', 2.500),
    (2, 1, 'end', 5.000)
]

activity_df = spark.createDataFrame(activity_data, ["machine_id", "process_id", "activity_type", "timestamp"])
activity_df.show()

print("""Output: 
+------------+-----------------+
| machine_id | processing_time |
+------------+-----------------+
| 0          | 0.894           |
| 1          | 0.995           |
| 2          | 1.456           |
+------------+-----------------+ 
Explanation: 
There are 3 machines running 2 processes each.
Machine 0's average time is ((1.520 - 0.712) + (4.120 - 3.140)) / 2 = 0.894
Machine 1's average time is ((1.550 - 0.550) + (1.420 - 0.430)) / 2 = 0.995
Machine 2's average time is ((4.512 - 4.100) + (5.000 - 2.500)) / 2 = 1.456""")

Average Time of Process per Machine
There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.
The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.
The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.
Input: 
+----------+----------+-------------+---------+
|machine_id|process_id|activity_type|timestamp|
+----------+----------+-------------+---------+
|         0|         0|        start|    0.712|
|         0|         0|          end|     1.52|
|         0|         1|        start|     3.14|
|         0|         1|          end|     4.12|
|         1|         0|        start|     0.55|
|         1|         0|      

In [37]:
#Group By Aproach
(
    activity_df.groupBy(F.col("machine_id"))
    .agg(
        F.sum(
            F.when(F.col("activity_type") == "start", F.col("timestamp")).otherwise(0)
        ).alias("start_timestamp"),
        F.sum(
            F.when(F.col("activity_type") == "end", F.col("timestamp")).otherwise(0)
        ).alias("end_timestamp"),
        F.count(
            F.when(F.col("activity_type") == "start", F.col("timestamp")).otherwise(
                None
            )
        ).alias("no_of_process"),
    )
    .select(
        F.col("machine_id"),
        F.round(
            (F.col("end_timestamp") - F.col("start_timestamp"))
            / F.col("no_of_process"),
            3,
        ).alias("processing_time "),
    )
    .show()
)

+----------+----------------+
|machine_id|processing_time |
+----------+----------------+
|         0|           0.894|
|         1|           0.995|
|         2|           1.456|
+----------+----------------+

